In [7]:
import xml.etree.ElementTree as ET

In [8]:
tree = ET.parse('../../Data/Datasets/Common/suc3train.xml')
root = tree.getroot()
spacyJson = []
ents = ["PRS", "ORG", "LOC", "TME", "MSR", "WRK", "EVN", "OBJ"]
sentsList = []

for sent in root.iter('sentence'):
    tokensList = []
    for child in sent:
        if child.tag == 'name':
            for token in child.iter('w'):
                nameDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                tokensList.append(nameDict)
        elif child.tag == 'ne' and child.attrib['type'] in ents:
            entityTokenList = child.attrib['name'].split(' ')
            for token in child.iter('w'):
                neDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|')}
                if len(entityTokenList) == 1:
                    neDict['ner'] = 'U-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
                elif token.text == entityTokenList[len(entityTokenList) - 1]:
                    neDict['ner'] = 'L-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
                elif token.text == entityTokenList[0]:
                    neDict['ner'] = 'B-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
                else:
                    neDict['ner'] = 'I-' + child.attrib['type']
                    tokensList.append(neDict)
        elif child.tag == 'ne':
            for token in child.iter('w'):
                tokenDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                tokensList.append(tokenDict)
        elif child.tag == 'w':
            tokenDict = {'orth': child.text, 'tag': child.attrib['msd'].replace('.', '|'), "ner": "O"}
            tokensList.append(tokenDict)
        else:
            continue
    sentsList.append(tokensList)


In [9]:

with open('../../Data/Datasets/ner/SUC3trainCONLL2003', 'w', encoding='utf-8') as f:
    f.write('-DOCSTART- -X- O O' + '\n\n')
    for sent in sentsList:
        for token in sent:
            f.write(token['orth'] + ' ' + token['tag'] + ' ' + 'O' + ' ' + token['ner'] + '\n')
        f.write('\n')